# Reading the Train File for en-us text

In [27]:
# Returns {Line : Article}

def readnormalcorpus():
    linetest=0
    corpus={}
    fi=open("normal_test.txt","r")
    for line in fi:
        linetest+=1
        if linetest > 100:
            break
        corpus[[((line[:-1].split("\t")))[2]][0]]=[((line[:-1].split("\t")))[0]][0];
    return corpus

# Reading Train File for Simple Text

In [29]:
# Returns {Line : Article}

def readsimplecorpus():
    linetest=0
    corpus={}
    fi=open("simple_test.txt","r")
    for line in fi:
        linetest+=1
        if linetest > 100:
            break
        corpus[[((line[:-1].split("\t")))[2]][0]]=[((line[:-1].split("\t")))[0]][0];
    return corpus

# Building a Trigram Model

In [54]:
# from itertools import tee, izip

def window(iterable, size):
    iters = tee(iterable, size)
    for i in xrange(1, size):
        for each in iters[i:]:
            next(each, None)
    return izip(*iters)

def tri_model():
    
    normal_corpus = readnormalcorpus()
    simple_corpus = readsimplecorpus()
    
    tgm_normal={}
    tgm_simple={}
        
    for i in normal_corpus.keys():
        for each in window(i.split(), 3):
            temp=list(each)
            if tgm_normal.has_key(tuple(temp)):
                tgm_normal[tuple(list(each))]+=1
            else:
                tgm_normal[tuple(list(each))]=1
    
                
    for i in simple_corpus.keys():
        for each in window(i.split(), 3):
            temp=list(each)
            if tgm_simple.has_key(tuple(temp)):
                tgm_simple[tuple(list(each))]+=1
            else:
                tgm_simple[tuple(list(each))]=1

    return tgm_normal,tgm_simple

# First, Defining a Naive Bias Model for Classification

This function takes a trigram input and does a simple vs normal classification. As the trigram which is normal is also present in simple we classify it in simple.

In [82]:
import numpy as np
from sklearn.naive_bayes import GaussianNB


def check_NB():
    
    X=[]
    Y=[]
    
    tgm_normal,tgm_simple = tri_model()
    
    # Building X
    X.extend(tgm_normal.keys())
    X.extend(tgm_simple.keys())
    
    X=np.array(X)
    
    # Building Y
    for i in tgm_normal.keys():
        Y.append(0)
    for i in tgm_simple.keys():
        Y.append(1)
    
    Y=np.array(Y)
    
    print X

    
    
    model = GaussianNB()
    model.fit(X, Y)
    

check_NB()

[['Numa' 'Pompilius' ',']
 ['may' 'be' 'to']
 ['years' 'February' 'starts']
 ...
 ['deleted' 'under' 'policy']
 ['emotion' 'and' 'feels']
 ['to' 'connect' 'with']]


TypeError: cannot perform reduce with flexible type

In [87]:
def naive_classifier():
    tgm_normal,tgm_simple=tri_model()
    sentence="The Venerable Bede says that this month is the root of the word Easter ."
    
    for each in window(sentence.split(), 3):
        temp=list(each)
        if(tgm_simple.has_key(tuple(temp))):
            score_simple*=tgm_simple(tuple(temp))
        else :
            score_simple*=0.0003
        if(tgm_normal.has_key(tuple(temp))):
            score_simple*=tgm_simple(tuple(temp))

       
    

({('Numa', 'Pompilius', ','): 1,
  ('may', 'be', 'to'): 1,
  ('years', 'February', 'starts'): 1,
  ('generate', 'strong', 'emotions'): 1,
  ('cultures', 'that', 'produced'): 1,
  ('considered', 'applied', 'art'): 1,
  ('a', 'way', 'that'): 1,
  ('as', 'discipline', '-RRB-'): 1,
  ('be', 'characterized', 'by'): 1,
  ('or', 'to', 'generate'): 1,
  (';', 'to', 'create'): 1,
  ('saw', 'a', 'host'): 1,
  ('because', 'so', 'little'): 1,
  ('container', ',', 'may'): 1,
  ('creator', ',', 'regardless'): 1,
  ('art', 'as', 'form'): 1,
  ('psychology', 'analyze', 'its'): 1,
  ('and', 'ideas', ','): 1,
  ('parameters', 'of', 'Impressionism'): 1,
  (',', 'artifice', ','): 1,
  ('a', 'veneration', 'of'): 1,
  ('By', 'this', 'definition'): 1,
  ('the', 'finer', 'things'): 1,
  ('pottery', '-LRB-', 'including'): 1,
  ('latter', 'half', 'of'): 1,
  ('or', '``', 'craft'): 1,
  ('20th', 'century', 'to'): 1,
  (',', 'much', 'art'): 1,
  ('during', 'the', 'Romantic'): 1,
  (',', 'people', 'will'): 1,
  ('

# Defining a logestic Regression Model

In [83]:
class LogisticRegression:
    def __init__(self, lr=0.01, num_iter=100000, fit_intercept=True, verbose=False):
        self.lr = lr
        self.num_iter = num_iter
        self.fit_intercept = fit_intercept
        self.verbose = verbose
    
    def __add_intercept(self, X):
        intercept = np.ones((X.shape[0], 1))
        return np.concatenate((intercept, X), axis=1)
    
    def __sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    def __loss(self, h, y):
        return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()
    
    def fit(self, X, y):
        if self.fit_intercept:
            X = self.__add_intercept(X)
        
        # weights initialization
        self.theta = np.zeros(X.shape[1])
        
        for i in range(self.num_iter):
            z = np.dot(X, self.theta)
            h = self.__sigmoid(z)
            gradient = np.dot(X.T, (h - y)) / y.size
            self.theta -= self.lr * gradient
            
            z = np.dot(X, self.theta)
            h = self.__sigmoid(z)
            loss = self.__loss(h, y)
                
            if(self.verbose ==True and i % 10000 == 0):
                print(f'loss: {loss} \t')
    
    def predict_prob(self, X):
        if self.fit_intercept:
            X = self.__add_intercept(X)
    
        return self.__sigmoid(np.dot(X, self.theta))
    
    def predict(self, X):
        return self.predict_prob(X).round()

SyntaxError: invalid syntax (<ipython-input-83-b7f9674b559b>, line 35)

In [86]:
# load the iris dataset 
from sklearn.datasets import load_iris 
iris = load_iris() 

# store the feature matrix (X) and response vector (y) 
X = iris.data 
y = iris.target 

# splitting X and y into training and testing sets 
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1) 

# training the model on training set 
from sklearn.naive_bayes import GaussianNB 
gnb = GaussianNB() 
gnb.fit(X_train, y_train) 

print X_train

# making predictions on the testing set 
y_pred = gnb.predict(X_test) 

# comparing actual response values (y_test) with predicted response values (y_pred) 
from sklearn import metrics 
print("Gaussian Naive Bayes model accuracy(in %):", metrics.accuracy_score(y_test, y_pred)*100)

[[4.8 3.4 1.6 0.2]
 [5.7 2.5 5.  2. ]
 [6.3 2.7 4.9 1.8]
 [4.8 3.  1.4 0.1]
 [4.7 3.2 1.3 0.2]
 [6.5 3.  5.8 2.2]
 [4.6 3.4 1.4 0.3]
 [6.1 3.  4.9 1.8]
 [6.5 3.2 5.1 2. ]
 [6.7 3.1 4.4 1.4]
 [5.7 2.8 4.5 1.3]
 [6.7 3.3 5.7 2.5]
 [6.  3.  4.8 1.8]
 [5.1 3.8 1.6 0.2]
 [6.  2.2 4.  1. ]
 [6.4 2.9 4.3 1.3]
 [6.5 3.  5.5 1.8]
 [5.  2.3 3.3 1. ]
 [6.3 3.3 6.  2.5]
 [5.5 2.5 4.  1.3]
 [5.4 3.7 1.5 0.2]
 [4.9 3.1 1.5 0.2]
 [5.2 4.1 1.5 0.1]
 [6.7 3.3 5.7 2.1]
 [4.4 3.  1.3 0.2]
 [6.  2.7 5.1 1.6]
 [6.4 2.7 5.3 1.9]
 [5.9 3.  5.1 1.8]
 [5.2 3.5 1.5 0.2]
 [5.1 3.3 1.7 0.5]
 [5.8 2.7 4.1 1. ]
 [4.9 3.1 1.5 0.1]
 [7.4 2.8 6.1 1.9]
 [6.2 2.9 4.3 1.3]
 [7.6 3.  6.6 2.1]
 [6.7 3.  5.2 2.3]
 [6.3 2.3 4.4 1.3]
 [6.2 3.4 5.4 2.3]
 [7.2 3.6 6.1 2.5]
 [5.6 2.9 3.6 1.3]
 [5.7 4.4 1.5 0.4]
 [5.8 2.7 3.9 1.2]
 [4.5 2.3 1.3 0.3]
 [5.5 2.4 3.8 1.1]
 [6.9 3.1 4.9 1.5]
 [5.  3.4 1.6 0.4]
 [6.8 2.8 4.8 1.4]
 [5.  3.5 1.6 0.6]
 [4.8 3.4 1.9 0.2]
 [6.3 3.4 5.6 2.4]
 [5.6 2.8 4.9 2. ]
 [6.8 3.2 5.9 2.3]
 [5.  3.3 1.